In [105]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train.csv', 'sample_submission.csv', 'test.csv']


In [106]:
train=pd.read_csv('../input/train.csv')
test=pd.read_csv('../input/test.csv')
print(train.info())
print(train.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB
None
   label  pixel0  pixel1    ...     pixel781  pixel782  pixel783
0      1       0       0    ...            0         0         0
1      0       0       0    ...            0         0         0
2      1       0       0    ...            0         0         0
3      4       0       0    ...            0         0         0
4      0       0       0    ...            0         0         0

[5 rows x 785 columns]


In [107]:
print(train.isnull().any().describe())
print(train.shape)

count       785
unique        1
top       False
freq        785
dtype: object
(42000, 785)


In [108]:
d=pd.get_dummies(train['label'])
names=['0','1','2','3','4','5','6','7','8','9']
c=0
for col in names:
    train[col]=d[c]
    c+=1
train.drop('label',axis=1,inplace=True)
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 794 entries, pixel0 to 9
dtypes: int64(784), uint8(10)
memory usage: 251.6 MB
None


In [109]:
X=train.drop(names,axis=1)
Y=train[names]

In [110]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.1)
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

(37800, 784) (37800, 10) (4200, 784) (4200, 10)


In [111]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Conv2D,MaxPooling2D,Flatten,GlobalAveragePooling2D
from keras.optimizers import SGD
model=Sequential()
model.add(Dense(64, activation='relu',input_shape=(784,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(10,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [112]:
hist=model.fit(x_train,y_train,epochs=10,batch_size=32)

Epoch 1/10
37800/37800 [==============================] - 9s 236us/step - loss: 1.0135 - acc: 0.8206
Epoch 2/10
37800/37800 [==============================] - 8s 202us/step - loss: 0.2450 - acc: 0.9294
Epoch 3/10
37800/37800 [==============================] - 8s 201us/step - loss: 0.1922 - acc: 0.9451
Epoch 4/10
37800/37800 [==============================] - 8s 203us/step - loss: 0.1640 - acc: 0.9524
Epoch 5/10
37800/37800 [==============================] - 8s 201us/step - loss: 0.1383 - acc: 0.9597
Epoch 6/10
37800/37800 [==============================] - 7s 197us/step - loss: 0.1221 - acc: 0.9644
Epoch 7/10
37800/37800 [==============================] - 7s 196us/step - loss: 0.1084 - acc: 0.9692
Epoch 8/10
37800/37800 [==============================] - 8s 210us/step - loss: 0.1008 - acc: 0.9713
Epoch 9/10
37800/37800 [==============================] - 7s 193us/step - loss: 0.0891 - acc: 0.9737
Epoch 10/10
37800/37800 [==============================] - 7s 194us/step - loss: 0.0817 - a

In [113]:
model.evaluate(x_test,y_test)

4200/4200 [==============================] - 1s 170us/step


[0.1350477811863779, 0.9654761904761905]

In [114]:
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [115]:
ans=model.predict(test)
ans=pd.DataFrame(ans)
ans=ans.idxmax(axis=1)
a=[]
id=[]
for i in range(0,28000):
    a.append(ans[:][i])
    id.append(i+1)

In [116]:
sub=pd.DataFrame({'ImageId':id,'Label':a})
sub.to_csv('output.csv',index=False)